In [1]:
import numpy as np
import cv2 # OpenCV biblioteka
import matplotlib
import matplotlib.pyplot as plt
import pylab
import imageio as img

In [2]:
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 
# prikaz vecih slika
matplotlib.rcParams['figure.figsize'] = 16,12
kernel = np.ones((3, 3))

In [3]:
with open("out.txt", "w") as resultFile:
    resultFile.write("RA205/2015, Luka Cvetinovic\n")
    resultFile.write("file,count\n")
for q in range(1, 11):
    filename = 'videos/video'+str(q)+'.mp4'
    vtype = 'ffmpeg'
    vid = img.get_reader(filename,  vtype)
    image = vid.get_data(0)
 
    #plt.imshow(image);
    #plt.show()
 
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    max_hex = 255
    image_invert = max_hex-image_gray
 
    #ADAPTIVE THRESHOLD
    thresh_val = 200

    ret, image_bin = cv2.threshold(image_invert, thresh_val, max_hex, cv2.THRESH_BINARY)
    image_bin = cv2.erode(image_bin, kernel, iterations=1)

 
 
 
    ####### searching for contours
    slika, contours, hierarchy = cv2.findContours(image_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
 
    contours_p = [] #ovde ce biti samo konture koje pripadaju bar-kodu
    for contour in contours: # za svaku konturu
        center, size, angle = cv2.minAreaRect(contour) # pronadji pravougaonik minimalne povrsine koji ce obuhvatiti celu konturu
        width, height = size
        visina = 50
        if height > visina: # kontura platoa
            contours_p.append(contour) # ova kontura je plato
 
 
    slika = image.copy()
    red = (255, 0, 0)
    cv2.drawContours(slika, contours_p, -1, red, 1)
 
 
    # determine the most extreme points along the contour
    c = max(contours_p, key=cv2.contourArea)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
 
 
    # draw extreme points of contour rectangle
    cv2.circle(slika, extLeft, 8, (0, 0, 255), -1)
    cv2.circle(slika, extRight, 8, (0, 255, 0), -1)
    cv2.circle(slika, extTop, 8, (255, 0, 0), -1)
    cv2.circle(slika, extBot, 8, (255, 255, 0), -1)
 
 
    #print("Y centar:") #y
    center_y = int(((extBot[1]-extTop[1])/2)+extTop[1])
    #print(center_y)
 
    #print("X centar:") #y
    center_x = int(((extRight[0]-extLeft[0])/2)+extLeft[0])
    # print(center_x)
 
 
    #draw central point inside plato contour
    #cv2.circle(slika, (center_x, center_y), 8, (255, 255, 255), -1)
 
    #rectangle inside scene which is our designated area of observation
    cv2.rectangle(slika, (0, center_y-28), (640, center_y+28), (255, 255, 255), 2)
    io = slika[center_y-28:center_y+28, :]
 
    #plt.imshow(io)
    #plt.show()
 
 
    j=0
    k=0
    #contouring cropped area of frame
    counter = 0
    for i, im in enumerate(vid):
    #for i in enumerate(vid):
        j+=1
        if(j==30):
            k+=1
            image = vid.get_data(i)
            cropped = image[center_y-28:center_y+28, 255:400]
 
            #plt.imshow(cropped);
            #plt.show()
 
            image_gray = cv2.cvtColor(cropped, cv2.COLOR_RGB2GRAY)
 
            image_invert = max_hex-image_gray
 
            #ADAPTIVE THRESHOLD
            #ret, image_bin = cv2.threshold(image_invert, 180, 255, cv2.THRESH_BINARY)
            image_bin = cv2.adaptiveThreshold(image_invert, max_hex, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 5)
            image_bin = cv2.dilate(image_bin, kernel, iterations=1)
            image_bin = cv2.erode(image_bin, kernel, iterations=2)
 
            #plt.imshow(image_bin, 'gray');
            #plt.show()
 
 
 
 
            ############# searching for pedestrians using circle contouring
            nova, contours, hierarchy = cv2.findContours(image_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            contours_pedestrians = [] #ovde ce biti samo konture koje pripadaju bar-kodu
            for contour in contours: # za svaku konturu
                (x,y),radius = cv2.minEnclosingCircle(contour) #circular contours
                center = (int(x),int(y)) #contour center
                radius = int(radius) #contour radius
                if radius < 33 and radius > 2: # radius of pedestrian contour
                    contours_pedestrians.append(contour) # adding pedestrian contour
 
 
            nova = cropped.copy()
            green = (0, 255, 0)
            cv2.drawContours(nova, contours_pedestrians, -1, green, 1)
 
#             plt.imshow(nova)
#             plt.show()
 
            #counting contours inside one frame
 
            counter += len(contours_pedestrians)
#             print("# %i" % k)
#             print('No. of detected pedestrians in current frame: %d' % len(contours_pedestrians))
            j=0
           
    with open("out.txt", "a") as resultFile:
        resultFile.write('video'+str(q)+'.mp4'+","+str(counter)+"\n")
 
    print('KLIP: %d' % q)
    print('SUM: %d' % counter)

KLIP: 1
SUM: 5
KLIP: 2
SUM: 28
KLIP: 3
SUM: 16
KLIP: 4
SUM: 24
KLIP: 5
SUM: 34
KLIP: 6
SUM: 26
KLIP: 7
SUM: 22
KLIP: 8
SUM: 14
KLIP: 9
SUM: 4
KLIP: 10
SUM: 27
